In [1]:
pip install opencv-python mediapipe


^C
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
pip install opencv-python mediapipe tensorflow scikit-learn

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.optimizers import Adam

data_dir = "dataset/"
categories = sorted(os.listdir(data_dir))

data = []
labels = []

for label, category in enumerate(categories):
    folder_path = os.path.join(data_dir, category)
    for img_name in os.listdir(folder_path):
        img_path = os.path.join(folder_path, img_name)
        img = cv2.imread(img_path)
        if img is not None:
            img = cv2.resize(img, (64, 64))
            data.append(img)
            labels.append(label)


X = np.array(data) / 255.0
y = to_categorical(labels, num_classes=len(categories))


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(64, 64, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(len(categories), activation='softmax')
])

model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

model.save("model/asl_model.h5")


Epoch 1/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step - accuracy: 0.0909 - loss: 3.3259 - val_accuracy: 0.0000e+00 - val_loss: 3.8938
Epoch 2/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 294ms/step - accuracy: 0.0455 - loss: 3.1809 - val_accuracy: 0.0000e+00 - val_loss: 4.4029
Epoch 3/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 280ms/step - accuracy: 0.1818 - loss: 3.0166 - val_accuracy: 0.0000e+00 - val_loss: 4.9151
Epoch 4/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 319ms/step - accuracy: 0.2273 - loss: 2.8437 - val_accuracy: 0.0000e+00 - val_loss: 5.4361
Epoch 5/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 270ms/step - accuracy: 0.3636 - loss: 2.5934 - val_accuracy: 0.0000e+00 - val_loss: 5.9492
Epoch 6/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 262ms/step - accuracy: 0.6818 - loss: 2.3043 - val_accuracy: 0.0000e+00 - val_loss: 6.4297
Epoch 7/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 271ms/step - accuracy: 0.7273 - loss: 1.9817 - val_accuracy: 0.0000e+00 - val_loss: 7.0539
Epoch 8/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 286ms/step - accuracy: 0.8636 - loss: 1.6583 - val_acc

In [2]:
import os
print(os.getcwd())


C:\Users\HARSHAVARDHAN\Desktop\sign lang


In [3]:
pip install tqdm


  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
Using cached tqdm-4.67.1-py3-none-any.whl (78 kB)
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [8]:
# train.py
import os
import cv2
import numpy as np
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
import tensorflow as tf

DATA_DIR = "dataset/"
CLASSES = [chr(i) for i in range(ord('A'), ord('Z') + 1)]
IMG_SIZE = 96
X, y = [], []

for idx, label in enumerate(CLASSES):
    path = os.path.join(DATA_DIR, label)
    for img in tqdm(os.listdir(path)[:1500], desc=label):  # Limit per class
        try:
            img_array = cv2.imread(os.path.join(path, img))
            img_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
            X.append(img_array)
            y.append(idx)
        except:
            continue

X = np.array(X) / 255.0
y = to_categorical(y, num_classes=len(CLASSES))

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

datagen = ImageDataGenerator(
    rotation_range=15,
    zoom_range=0.2,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True
)
datagen.fit(X_train)

base_model = MobileNetV2(input_shape=(IMG_SIZE, IMG_SIZE, 3), include_top=False, weights='imagenet')
base_model.trainable = False

model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(len(CLASSES), activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(datagen.flow(X_train, y_train, batch_size=32), validation_data=(X_test, y_test), epochs=5)

model.save('model/asl_model.h5')
print("✅ Model trained and saved.")


Z: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1500/1500 [00:02<00:00, 697.67it/s]


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 13s 1us/step


C:\Users\HARSHAVARDHAN\Desktop\sign lang\venv\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/5
975/975 ━━━━━━━━━━━━━━━━━━━━ 125s 121ms/step - accuracy: 0.3658 - loss: 2.1565 - val_accuracy: 0.8606 - val_loss: 0.4674
Epoch 2/5
975/975 ━━━━━━━━━━━━━━━━━━━━ 98s 100ms/step - accuracy: 0.6754 - loss: 0.9868 - val_accuracy: 0.8969 - val_loss: 0.3347
Epoch 3/5
975/975 ━━━━━━━━━━━━━━━━━━━━ 96s 99ms/step - accuracy: 0.7190 - loss: 0.8432 - val_accuracy: 0.9124 - val_loss: 0.2645
Epoch 4/5
975/975 ━━━━━━━━━━━━━━━━━━━━ 99s 101ms/step - accuracy: 0.7483 - loss: 0.7554 - val_accuracy: 0.9167 - val_loss: 0.2482
Epoch 5/5
975/975 ━━━━━━━━━━━━━━━━━━━━ 98s 101ms/step - accuracy: 0.7614 - loss: 0.7197 - val_accuracy: 0.9171 - val_loss: 0.2461


✅ Model trained and saved.
